# Lab One
### Eric Smith, Tyler Giallanza, Oscar Vallner

## DeepDream Implementation

In this section we will implement DeepDream using Keras. We will start off with an implementation of InceptionV3 that has been pretrained on the ImageNet dataset. This code is heavily based on Francois Chollet's implementation provided in the Deep Learning with Python book and implemented [here](https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/8.2-deep-dream.ipynb).

In [1]:
from keras.applications import inception_v3
from keras import backend as K

#We are using a pretrained model and thus should disable learning functions
K.set_learning_phase(0)

#Load the pre-trained inceptionV3 model based on the imagenet dataset
model = inception_v3.InceptionV3(weights='imagenet',
                                 include_top=False)

Using TensorFlow backend.


87916544/87910968 [==============================] - 38s 0us/step


In [2]:
print([layer.name for layer in model.layers if layer.name.startswith('mixed')])

['mixed0', 'mixed1', 'mixed2', 'mixed3', 'mixed4', 'mixed5', 'mixed6', 'mixed7', 'mixed8', 'mixed9_0', 'mixed9', 'mixed9_1', 'mixed10']


Next, we use gradient ascent to iteratively update the input image. Unlike the example notebook from class that updated the input image to maximize the activation of a single filter, here we update the input image to maximize the activations of all filters in a given layer. Furthermore, we include multiple layers that are weighted differently.

Earlier model layers detect features at a lower level of abstraction, and deeper layers detect higher levels of abstraction. For example, the first few layers likely detect edges and textures, whereas later layers likely detect class-relevant information, such as the presence of ears in a dog. Because we want varied levels of abstraction in our output image, we include layers from earlier in the network as well as layers from later in the network.

The loss for our gradient ascent is comprised of a weighted sum for the losses of each layer we are examining. Each layer's loss is the L2 norm - the average sum of squares for each filter activation in the layer.

In [16]:
#This defines the amount that each chosen layer contributes to the visualization
#Note that layer names are pre-determined by the already created InceptionV3 model
layer_contributions = {
    'mixed2': 0.2,
    'mixed3': 3.,
    'mixed4': 2.,
    'mixed5': 1.5,
}

#Map each layer object to the name of the layer - this allows us to access the layers we defined in the
#layer contributions dictionary
layer_dict = dict([(layer.name, layer) for layer in model.layers])

#Define the loss function
loss = K.variable(0.)
for layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output
    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    #The loss for each layer is the L2 norm - we ignore the border pixels of each layer to avoid artifacts
    #present at the borders
    loss = loss + coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling

With the loss function defined, we can now code the gradient ascent algorithm. Notice that we normalize the gradients at each step with the L1 norm. This step is a "trick" that helps training FOR SOME REASON.

In [10]:
#Layer containing the image we are iteratively updating
dream = model.input

#Gradients for the image w.r.t. the loss function
grads = K.gradients(loss, dream)[0]

#L1 gradient normalization trick - add epsilon value to prevent zero division
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)


#Return the loss and the gradients by inputing the generated image to the model
outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)

#Call the function that gets the model loss and gradients
def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values

#Run the gradient ascent for a given number of iterations and a given step size (learning rate)
def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('...Loss value at', i, ':', loss_value)
        x += step * grad_values
    return x

In [11]:
import scipy
from keras.preprocessing import image

#Resizes a given image by zooming
def resize_img(img, size):
    img = np.copy(img)
    factors = (1,
               float(size[0]) / img.shape[1],
               float(size[1]) / img.shape[2],
               1)
    return scipy.ndimage.zoom(img, factors, order=1)

#Saves an image to disk
def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    scipy.misc.imsave(fname, pil_img)

#Loads images from disk, resizing and formatting them into a format processable by InceptionV3
def preprocess_image(image_path):
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img

#Converts tensors into an image that can be visualized with matplotlib
def deprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [20]:
import numpy as np
import matplotlib.pyplot as plt


step = 0.02  # Gradient ascent step size
num_octave = 7  # Number of scales at which to run gradient ascent
octave_scale = 1.4  # Size ratio between scales
iterations = 50  # Number of ascent steps per scale

# If our loss gets larger than 10,
# we will interrupt the gradient ascent process, to avoid ugly artifacts
max_loss = 10.

# Path of the original image fed into DeepDream
base_image_path = 'ocean.jpg'

# Format the image so it is interpretable by InceptionV3
img = preprocess_image(base_image_path)

# Determine the different scales for image resizing
original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
    successive_shapes.append(shape)

# Reverse list of shapes, so that they are in increasing order
successive_shapes = successive_shapes[::-1]

# Resize the Numpy array of the image to our smallest scale
original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

#For every shape run gradient ascent for the given number of iterations
for shape in successive_shapes:
    print('Processing image shape', shape)
    img = resize_img(img, shape)
    img = gradient_ascent(img,
                          iterations=iterations,
                          step=step,
                          max_loss=max_loss)
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    same_size_original = resize_img(original_img, shape)
    lost_detail = same_size_original - upscaled_shrunk_original_img

    img += lost_detail #Trick to add back the missing information when resizing
    shrunk_original_img = resize_img(original_img, shape)
    save_img(img, fname='dream_at_scale_' + str(shape) + '.png')
    plt.show()

save_img(img, fname='final_dream.png')

Processing image shape (344, 516)
...Loss value at 0 : 0.360418
...Loss value at 1 : 0.505955
...Loss value at 2 : 0.627866
...Loss value at 3 : 0.662679
...Loss value at 4 : 0.803686
...Loss value at 5 : 0.942463
...Loss value at 6 : 1.04801
...Loss value at 7 : 1.17987
...Loss value at 8 : 1.06032
...Loss value at 9 : 1.36917
...Loss value at 10 : 1.50229
...Loss value at 11 : 1.74531
...Loss value at 12 : 1.68397
...Loss value at 13 : 1.9189
...Loss value at 14 : 2.37454
...Loss value at 15 : 2.14165
...Loss value at 16 : 2.6612
...Loss value at 17 : 2.99019
...Loss value at 18 : 3.09037
...Loss value at 19 : 3.32936
...Loss value at 20 : 3.70507
...Loss value at 21 : 3.67409
...Loss value at 22 : 4.49966
...Loss value at 23 : 4.71485
...Loss value at 24 : 5.49345
...Loss value at 25 : 5.45942
...Loss value at 26 : 5.61378
...Loss value at 27 : 6.31721
...Loss value at 28 : 6.70549
...Loss value at 29 : 6.82942
...Loss value at 30 : 7.37033
...Loss value at 31 : 7.97153
...Loss valu

KeyboardInterrupt: 